# Introduction


## Challenge Large Scale Machine Learning

### Authors: 
#### Kevin Elgui (kevin.elgui@gmail.com), Pascal Bianchi



### Geolocation challenge

The goal of this challenge is to develop a system which tries to perform a geolocation of connected devices within an IoT Network.

The challenge will have two phases. Please read the below information carefully.

In the first phase of the challenge, you will be provided a training dataset which contains the raw messages and their corresponding coordinates. We will also provide you a validation dataset, which only contains the raw messages. The labes of the validation set will not be provided. In this phase, you will train your system based on this data and the ranking in the challenge webpage will be based on your score obtained on the validation data.

The second phase of the challenge will start only a couple of hours before the deadline. Within this period, we will provide you the test set. You will need to run the algorithm that you developed on the first phase on this dataset and submit your predictions to the challenge webpage. Your final ranks will be determined on this dataset. Note that this phase will only last a couple of hours, which means that you will not be able to tune your algorithm on this dataset.

# The properties of the dataset:


### Training data:


The training set contains raw messages (lines) and their labels (GPS-position). There are ?? messages in this dataset. Each message is of size 1 x 20, where 20 is the number of message descriptors. For each message there is a label, a couple (latitude, longitude). 

### Validation data:



The validation set only contains raw messages. There are ?? messages in this dataset, where the format is the same as before. The labels of this dataset will not be provided. Within the first phase, your ranks will be calculated based on this dataset.

### Test data:


The test set only contains raw messages as well. There will be ?? messages in this dataset, the format is always the same. The labels of this dataset will not be provided either. This dataset will be provided only in the second phase: only a couple of hours before the challenge deadline. Your final ranks will be calculated based on this dataset.

## The performance criterion¶

The question of a suitable criterion for evaluation is tough. In practice, in order to suit industrial requirements, we evaluate your prediction throught the 90% quantile of the error vector, which is computed as follows:
$$ error = d_V(pred, true),$$ where $d_V$ is the Vincenty distance between to GPS-coordinates. Finally:

$$ criterion =  error_{(90\%)}$$
In other words, this criterion corresponds to the max error distance made by your model on $90\%$ of the test set.
The lower the score, the better the performance.

# Training Data

Raw messages: lien_to_raw_messages_train

GPS-coordinates: lien_to_label_train

# Validation Data 

Raw messages: lien_to_raw_messages_val

# Example submission

In [ ]:
%matplotlib inline
import numpy as np
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
from geopy.distance import vincenty

## Loading and investigate the data

In [ ]:
toy_messages_fname = '../Data/Toy/db_toy.csv'
assert(os.path.exists(toy_messages_fname))
toy_labels_fname = '../Data/Toy/labels_toy.csv'
assert(os.path.exists(toy_labels_fname))

toy_messages_df = pd.read_csv(toy_messages_fname, sep=';', low_memory=False)
toy_labels_df = pd.read_csv(toy_labels_fname, sep=';', low_memory=False)

#toy_messages_fname.head()
toy_messages_df.head()

In [ ]:
toy_messages_df.shape

In [ ]:
toy_labels_df.head()

In [ ]:
toy_labels_df.shape

In [ ]:
train_messages_fname = '../Data/Train/db_train.csv'
assert(os.path.exists(train_messages_fname))
train_labels_fname = '../Data/Train/label_train.csv'
assert(os.path.exists(train_labels_fname))
val_messages_fname  = '../Data/Validation/db_val.csv'
assert(os.path.exists(val_messages_fname))
val_labels_fname = '../Data/Validation/label_val.csv'

train_messages_df = pd.read_csv(train_messages_fname, sep=';', low_memory=False)
train_labels_df = pd.read_csv(train_labels_fname, sep=';', low_memory=False)

train_messages_fname.head()


### Data Visualization (requires bokeh)


In [ ]:
## If you have bokeh installed, you can uncomment the last commands. 

sys.path.append('../Utils/')

#from plot_map import PlotMap
#PlotMap(toy_messages_df)

## Example prediction

In the following, we will train a simple regressor on toy data.

In [ ]:
toy_data = pd.read_csv('../Data/Toy/db_toy.csv', sep=';')
toy_labels = pd.read_csv('../Data/Toy/labels_toy.csv', sep=';', usecols=['latitude', 'longitude'] )

In [ ]:
toy_data.shape

In [ ]:
toy_data['messageid'].nunique()

In [ ]:
toy_data['time_ux'][0]-toy_data['time_ux_client'][0]#重要

In [ ]:
cl = toy_data.groupby('bsid')
cl.size()

### Features selection

In this example, we propose a simple feature matrix $X$ such that :

$$\begin{cases} X[i,j] &= 1 &\text{ if the $j^{th}$ basestation received the $i^{th}$ message}, \\
 X[i,j] &= 0, &\text{otherwise}. \end{cases} $$

For the project, you are free to chose other parameters for your model.

In [ ]:
## Please run the cell
## --------------------

features_of_interest = {"features_of_interest": ["rssi"],
"target":["latitude", "longitude"]}

dict_of_gby = {'rssi': ['bsid'],
                'freq': ['bsid'],
                'latitude_bs': ['bsid'],
                'longitude_bs': ['bsid'],
                'latitude': [''],
                'longitude': [''],
                'speed': [''],
                'dtid': [''], 
                'did': [''],
              }

In [ ]:
%%time

from features_builder import _build_features_dict, my_parser
(feature_dict, feature_name), (target_dict, target_name)  = _build_features_dict(toy_data, features_of_interest)
res = my_parser(toy_data, toy_labels, feature_dict, target_dict, features_of_interest)
X = pd.DataFrame(res['X'].toarray(), columns=res['dict_X'])
y = pd.DataFrame(res['y'].toarray(), columns=res['dict_y'])



In [ ]:
X.shape

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from evaluation import *

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)  

pred = pd.DataFrame(reg.predict(X_test), columns=res['dict_y'])
#print('The criterion on the testing data : %.2f km' % criterion(pred, y_test))

#np.savetxt('../Data/Toy/pred_toy.txt', pred.values)

In [ ]:
plot_error(pred.values, y_test)

#### Remark: 
The criterion is about 20 km. It's huge ! 

### Notation 
After having upload your file pred_test.txt on the server, you will be ranked w.r.t the criterion. 
The score which is assigned to you is computed on half of the test set. This corresponds to the public score. 
On the other half, you are assigned a private score. 

Your final score will take into account these two scores. Thus, your final rank may differ slightly from the one you see on the leader board.

#### Now it's your turn. Good luck !  :) 